In [1]:
import pandas as pd
import numpy as np
import time
import random
import json
import operator
import pycountry
import pycountry_convert
from iso3166 import countries

### Unesco Demographic Information

In [2]:
unesco_demo = pd.read_table("Data/unesco_demo.csv", sep = "\t")
unesco_demo.head()

/home/sara/myenv/lib64/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DEMO_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2003,2003,1.748,NaN,NaN
1,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2004,2004,1.768,NaN,NaN
2,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2005,2005,1.807,NaN,NaN
3,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2006,2006,1.908,NaN,NaN
4,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2007,2007,1.959,NaN,NaN


"DEMO_IND" and "Indicator" represent the same information? Let's see if there is a unique relation between the two fields... same for "TIME" and "Time".

In [3]:
demo_ind_relation = list(set([True for d in list(set(unesco_demo["DEMO_IND"])) if len(list(set(unesco_demo[unesco_demo["DEMO_IND"] == d]["Indicator"].values))) == 1]))
demo_ind_relation

[True]

In [4]:
demo_indicator = {d: unesco_demo[unesco_demo["DEMO_IND"]==d]["Indicator"].values[0] for d in list(set(unesco_demo["DEMO_IND"]))}
demo_indicator

{'200101': 'Total population ',
 '200144': 'Population aged 15-24 years ',
 '200151': 'Population aged 65 years or older ',
 '200343': 'Population aged 14 years or younger ',
 '200345': 'Population aged 25-64 years ',
 'DT_TDS_DECT_GN_ZS': 'Total debt service (% of GNI)',
 'NY_GDP_DEFL_ZS': 'GDP deflator (base year varies by country)',
 'NY_GDP_MKTP_CD': 'GDP (current US$)',
 'NY_GDP_MKTP_CN': 'GDP (current LCU)',
 'NY_GDP_MKTP_KD': 'GDP at market prices (constant 2010 US$)',
 'NY_GDP_MKTP_KD_ZG': 'GDP growth (annual %)',
 'NY_GDP_MKTP_KN': 'GDP (constant LCU)',
 'NY_GDP_MKTP_PP_CD': 'GDP, PPP (current international $)',
 'NY_GDP_MKTP_PP_KD': 'GDP, PPP (constant 2011 international $)',
 'NY_GDP_PCAP_CD': 'GDP per capita (current US$)',
 'NY_GDP_PCAP_CN': 'GDP per capita (current LCU)',
 'NY_GDP_PCAP_PP_CD': 'GDP per capita, PPP (current international $)',
 'NY_GDP_PCAP_PP_KD': 'GDP per capita, PPP (constant 2011 international $)',
 'NY_GNP_MKTP_CN': 'GNI (current LCU)',
 'NY_GNP_PCAP_C

In [8]:
time_relation = list(set([True for d in list(set(unesco_demo["TIME"])) if len(list(set(unesco_demo[unesco_demo["TIME"] == d]["Time"].values))) == 1]))
time_relation

[True]

In [9]:
temp_flag = list(set(unesco_demo["Flag Codes"]))
temp_flag = [i for i in temp_flag if str(i) != 'nan']
flag_demo = {f: unesco_demo[unesco_demo["Flag Codes"]==f]["Flags"].values[0]for f in temp_flag}
flag_demo

{'+': 'National Estimation',
 'n': 'Magnitude nil or negligible',
 '‡': 'UIS Estimation'}

No reasone for keeping:
- both the fields "DEMO_IND" and "Indicator", Just keep "DEMO_IND" and the dictionary demo_indicator.
- both the fields "TIME" and "Time"... same info!
- both the fields "Flag Codes" and "Flags"... same info! Just keep "Flags code" and the dictionary flag.
- both the fields "LOCATION" and "Country"... same info! Just keep "LOCATION".

Let's change the name:
- "LOCATION" --> "iso3"... coherence with other tables.
- "Flags code" --> "Flag"... more readable.
- "DEMO_IND" --> "Indicator"... more readable.

In [10]:
unesco_demo.head()

,DEMO_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2003,2003,1.748,NaN,NaN
1,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2004,2004,1.768,NaN,NaN
2,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2005,2005,1.807,NaN,NaN
3,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2006,2006,1.908,NaN,NaN
4,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,2007,2007,1.959,NaN,NaN


In [11]:
unesco_demo = unesco_demo[["DEMO_IND", "LOCATION", "Time", "Value", "Flag Codes"]]
unesco_demo.rename(columns = {"DEMO_IND": "Indicator", "LOCATION": "iso3", "Time": "Year","Flag Codes": "Flag"}, inplace=True)
unesco_demo.head()

/home/sara/myenv/lib64/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Indicator,iso3,Year,Value,Flag
0,SP_DYN_TFRT_IN,AUS,2003,1.748,NaN
1,SP_DYN_TFRT_IN,AUS,2004,1.768,NaN
2,SP_DYN_TFRT_IN,AUS,2005,1.807,NaN
3,SP_DYN_TFRT_IN,AUS,2006,1.908,NaN
4,SP_DYN_TFRT_IN,AUS,2007,1.959,NaN


### Problem!
Unesco table doe


### Unesco Education Information

In [63]:
unesco_education = pd.read_table("Data/unesco_education.csv", sep = "\t")
unesco_education.head()

,EDULIT_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2005,2005,19.49419,NaN,NaN
1,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2006,2006,18.20170,NaN,NaN
2,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2007,2007,17.86382,NaN,NaN
3,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2010,2010,17.25081,NaN,NaN
4,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2011,2011,20.31364,NaN,NaN


Same processing...

In [64]:
education_indicator = {d: unesco_education[unesco_education["EDULIT_IND"]==d]["Indicator"].values[0] for d in list(set(unesco_education["EDULIT_IND"]))}
education_indicator

{'XGOVEXP_IMF': 'Expenditure on education as a percentage of total government expenditure (%)',
 'XGOVEXP_IMF_02': 'Expenditure on pre-primary as a percentage of total government expenditure (%)',
 'XGOVEXP_IMF_1': 'Expenditure on primary as a percentage of total government expenditure (%)',
 'XGOVEXP_IMF_2': 'Expenditure on lower secondary as a percentage of total government expenditure (%)',
 'XGOVEXP_IMF_2T3': 'Expenditure on secondary as a percentage of total government expenditure (%)',
 'XGOVEXP_IMF_2T4_V': 'Expenditure on secondary and post-secondary non-tertiary vocational education as a percentage of total government expenditure (%)',
 'XGOVEXP_IMF_3': 'Expenditure on upper secondary as a percentage of total government expenditure (%)',
 'XGOVEXP_IMF_4': 'Expenditure on post-secondary non-tertiary as a percentage of total government expenditure (%)',
 'XGOVEXP_IMF_5T8': 'Expenditure on tertiary as a percentage of total government expenditure (%)'}

In [65]:
flag_education = {f: unesco_education[unesco_education["Flag Codes"]==f]["Flags"].values[0] if len(unesco_education[unesco_education["Flag Codes"]==f]["Flags"].values) != 0 else f for f in list(set(unesco_education["Flag Codes"]))}
flag_education

{'+': 'National Estimation',
 'a': 'Category not applicable',
 'n': 'Magnitude nil or negligible',
 nan: nan,
 '‡': 'UIS Estimation',
 '‡; n': 'UIS Estimation; Magnitude nil or negligible'}

In [66]:
unesco_education.head()

,EDULIT_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2005,2005,19.49419,NaN,NaN
1,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2006,2006,18.20170,NaN,NaN
2,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2007,2007,17.86382,NaN,NaN
3,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2010,2010,17.25081,NaN,NaN
4,XGOVEXP_IMF,Expenditure on education as a percentage of to...,BFA,Burkina Faso,2011,2011,20.31364,NaN,NaN


In [67]:
unesco_education= unesco_education[["EDULIT_IND", "LOCATION", "Time", "Value", "Flag Codes"]]
unesco_education.rename(columns = {"EDULIT_IND": "Indicator", "LOCATION": "iso3", "Time": "Year","Flag Codes": "Flag"}, inplace=True)
unesco_education.head()

,Indicator,iso3,Year,Value,Flag
0,XGOVEXP_IMF,BFA,2005,19.49419,NaN
1,XGOVEXP_IMF,BFA,2006,18.20170,NaN
2,XGOVEXP_IMF,BFA,2007,17.86382,NaN
3,XGOVEXP_IMF,BFA,2010,17.25081,NaN
4,XGOVEXP_IMF,BFA,2011,20.31364,NaN


### Cepii Distances

Geo-distances variables:
- iso_o, iso_d: ISO codes in three characters;
- contig: dummy, whether the two countries are contiguous;
- comlang_off: dummy, whether the two countries share the same official language;
- comcol: dummy, whether the two countries have had a common colonizer after 1945;
- colony: dummy, whether the two countries have had a colonial link;
- col45: dummy, whether the two countries have had a colonial relationship after tionship 1945;
- curcol: dummy, whether the two countries are currently in a colonial relationship;
- smctry: dummy, whether the two countries were/are the same country;
- dist: simple distance between the most important cities (in terms of population) of the two countries;
- distcap: simple distance between the capital cities of the two countries;
- distw, distwces: weighted distances; distance between the largest cities of the two countries - those inter cities distances are weighted by the share of the city in the overall country's population.


We don't really care about the "distcap".. "dist" variable is much more interesting! 

In [68]:
# country specific table
geo_info = pd.read_table("Data/cepii_geo_info.tsv", sep = "\t")
geo_info = geo_info[['iso2', 'iso3', 'country', 'area', 'dis_int', 'landlocked', 'continent', 'maincity', 'citynum',
                     'langoff_1', 'langoff_2', 'langoff_3', 'lang20_1', 'lang20_2', 'lang20_3', 'lang20_4', 'lang9_1', 
                     'lang9_2', 'lang9_3', 'lang9_4', 'colonizer1', 'colonizer2', 'colonizer3','colonizer4', 
                     'short_colonizer1', 'short_colonizer2','short_colonizer3']]
# it's not necessary to keep both iso2 and iso3.. same information! keep just iso3
geo_info = geo_info.drop("iso2", axis = 1)

# numeric type cols
geo_info["dis_int"] = [float(geo_info["dis_int"][i].replace(",", ".")) for i in geo_info.index]
geo_info["area"] = geo_info["area"].astype(int)

geo_info.head()

,iso3,country,area,dis_int,landlocked,continent,maincity,citynum,langoff_1,langoff_2,...,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
0,ABW,Aruba,193,5.225315,0,America,1,2,Dutch,Spanish,...,.,.,.,NLD,.,.,.,.,.,.
1,AFG,Afghanistan,652225,303.761400,1,Asia,1,25,Persian,.,...,Uzbek,.,.,.,.,.,.,GBR,.,.
2,AGO,Angola,1246700,419.966600,0,Africa,1,25,Portuguese,.,...,.,.,.,PRT,.,.,.,.,.,.
3,AIA,Anguilla,102,3.798690,0,America,1,2,English,.,...,.,.,.,GBR,.,.,.,.,.,.
4,ALB,Albania,28748,63.773110,0,Europe,1,25,Albanian,.,...,.,.,.,TUR,.,.,.,.,.,.


In [69]:
# pair of countries distances table
geo_dist = pd.read_table("Data/cepii_geo_dist.tsv", sep = "\t", na_values = {"distw": '.', "distwces": '.'})
geo_dist = geo_dist[['iso_o', 'iso_d', 'contig', 'dist', 'distcap', 'distw', 'distwces']]
geo_dist = geo_dist.drop("distcap", axis = 1)

# numeric type cols
geo_dist["dist"] = [float(geo_dist["dist"][i].replace(",", ".")) for i in geo_dist.index]
geo_dist["distw"] = [float(geo_dist["distw"][i].replace(",", ".")) if geo_dist["distw"][i] is not np.nan else np.nan for i in geo_dist.index]
geo_dist["distwces"] = [float(geo_dist["distwces"][i].replace(",", ".")) if geo_dist["distwces"][i] is not np.nan else np.nan for i in geo_dist.index]

geo_dist.head()

,iso_o,iso_d,contig,dist,distw,distwces
0,ABW,ABW,0,5.225315,25.09354,23.04723
1,ABW,AFG,0,13257.810000,13168.22000,13166.37000
2,ABW,AGO,0,9516.913000,9587.31600,9584.19300
3,ABW,AIA,0,983.268200,976.89740,976.89160
4,ABW,ALB,0,9091.742000,9091.57600,9091.46600


geo_dist table contains :
- duplicate information, e.g. ABW - AFG and AFG - ABW.
- info about the same countrie, e.g. ABW - ABW.

Actually I only care about distances from Italy!

In [70]:
geo_dist = geo_dist[(geo_dist["iso_o"] == "ITA") & (geo_dist["iso_d"] != "ITA")]
geo_dist.index = range(len(geo_dist))

In [71]:
geo_dist.head()

,iso_o,iso_d,contig,dist,distw,distwces
0,ITA,ABW,0,8480.1330,8427.7890,8422.343
1,ITA,AFG,0,4935.9940,4878.2850,4866.300
2,ITA,AGO,0,5645.2430,5850.5380,5827.607
3,ITA,AIA,0,7496.8750,7451.8810,7445.702
4,ITA,ALB,0,611.7626,722.9528,638.088


In [72]:
# Save it
geo_info.to_csv("Data/cepii_geo_info_norm.csv", sep = "\t", index = False)
geo_dist.to_csv("Data/cepii_geo_dist_norm.csv", sep = "\t", index = False)
unesco_demo.to_csv("Data/unesco_demo_norm.csv", sep = "\t", index = False)
unesco_education.to_csv("Data/unesco_education_norm.csv", sep = "\t", index = False)